In [ ]:
import pandas as pd
import numpy as np
import os
import re
from functools import partial
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import random
from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.initializers import LecunUniform
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import (Input, Conv2D, AveragePooling2D,
                                    UpSampling2D, concatenate, Conv2DTranspose,
                                    BatchNormalization,Concatenate)
from tensorflow.keras.callbacks import (Callback,EarlyStopping,
                                        ReduceLROnPlateau,ModelCheckpoint)

In [ ]:
def convolution_block(block_input : tf.float32,
                      num_filters : int = 256,
                      kernel_size :int = 3,
                      dilation_rate :int = 1,
                      padding : str = 'same',
                      use_bias : bool = False) -> tf.float32:
    x = Conv2D(num_filters,
               kernel_size = kernel_size,
               dilation_rate = dilation_rate,
               padding = padding ,
               use_bias = use_bias,
               kernel_initializer = LecunUniform())(block_input)
    x = BatchNormalization()(x)
    return tf.nn.relu(x)

def DilatedSpatialPyramidPooling(dspp_input : tf.float32) -> tf.float32:
    dims = dspp_input.shape
    x = AveragePooling2D(pool_size = (dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size = 1, use_bias = True)
    out_pool = UpSampling2D(size = (dims[-3] // x.shape[1],
                                    dims[-2] // x.shape[2]), 
                            interpolation = 'bilinear')(x)
    out_1 = convolution_block(dspp_input, 
                              kernel_size = 1,
                              dilation_rate = 1)
    out_6 = convolution_block(dspp_input,
                              kernel_size = 3,
                              dilation_rate = 6)
    out_12 = convolution_block(dspp_input,
                               kernel_size = 3 ,
                               dilation_rate = 12)
    out_18 = convolution_block(dspp_input,
                               kernel_size = 3,
                               dilation_rate = 18)
    x = Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    return convolution_block(x, kernel_size = 1)


def DeeplabV3Plus(height : int,
                  width : int,
                  color_channels : int,
                  num_classes : int) -> tf.float32:
    input_shape = (height, width, color_channels)
    model_input = Input(shape = input_shape)
    resnet50 = ResNet50(weights = 'imagenet',
                        include_top = False,
                        input_tensor = model_input)
    x = resnet50.get_layer('conv2_block2_2_relu').output
    x = DilatedSpatialPyramidPooling(x)
    

    input_a = UpSampling2D(size = (height // 4 // x.shape[1],
                                   width // 4 // x.shape[2]),
                           interpolation = 'bilinear')(x)
    input_b = resnet50.get_layer('conv2_block3_2_relu').output
    input_b = convolution_block(input_b, 
                                num_filters = 48,
                                kernel_size = 1)
    x = Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = UpSampling2D(size = (height // x.shape[1],
                             width // x.shape[2]),
                     interpolation = 'bilinear')(x)
    model_output = Conv2D(num_classes,
                          kernel_size = (1, 1),
                          padding = 'same')(x)
    return Model(inputs = model_input, outputs = model_output , name = 'DeepLabV3')

In [ ]:
def jacard_coef(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)